# Diversity with sgkit

In [1]:
%run setup.ipynb

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from dask.diagnostics import ProgressBar
import sgkit as sg
import xarray as xr

In [4]:
ds = xr.open_zarr(str(here() / 'data/sgkit/ag1000g_rechunked4.zarr'), concat_characters=False)
ds

<xarray.Dataset>
Dimensions:             (alleles: 4, ploidy: 2, samples: 1142, variants: 57837885)
Dimensions without coordinates: alleles, ploidy, samples, variants
Data variables:
    call_genotype       (variants, samples, ploidy) int8 dask.array<chunksize=(65536, 1142, 2), meta=np.ndarray>
    call_genotype_mask  (variants, samples, ploidy) bool dask.array<chunksize=(65536, 1142, 2), meta=np.ndarray>
    sample_id           (samples) <U1 dask.array<chunksize=(1142,), meta=np.ndarray>
    variant_allele      (variants, alleles) |S1 dask.array<chunksize=(65536, 4), meta=np.ndarray>
    variant_contig      (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_position    (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
Attributes:
    contigs:                    ['2L', '2R', '3L', '3R', 'X']
    max_variant_allele_length:  1

In [5]:
cac = xr.open_zarr(str(here() / 'data/sgkit/ag1000g_cohort_allele_count.zarr'), concat_characters=False)
cac

<xarray.Dataset>
Dimensions:              (alleles: 4, cohorts: 15, variants: 57837885)
Dimensions without coordinates: alleles, cohorts, variants
Data variables:
    cohort_allele_count  (variants, cohorts, alleles) int32 dask.array<chunksize=(65536, 15, 4), meta=np.ndarray>

In [6]:
ds = xr.merge([ds, cac])
ds

<xarray.Dataset>
Dimensions:              (alleles: 4, cohorts: 15, ploidy: 2, samples: 1142, variants: 57837885)
Dimensions without coordinates: alleles, cohorts, ploidy, samples, variants
Data variables:
    call_genotype        (variants, samples, ploidy) int8 dask.array<chunksize=(65536, 1142, 2), meta=np.ndarray>
    call_genotype_mask   (variants, samples, ploidy) bool dask.array<chunksize=(65536, 1142, 2), meta=np.ndarray>
    sample_id            (samples) <U1 dask.array<chunksize=(1142,), meta=np.ndarray>
    variant_allele       (variants, alleles) |S1 dask.array<chunksize=(65536, 4), meta=np.ndarray>
    variant_contig       (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_position     (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    cohort_allele_count  (variants, cohorts, alleles) int32 dask.array<chunksize=(65536, 15, 4), meta=np.ndarray>

In [7]:
ds = sg.window(ds, size=200, step=200)
ds

<xarray.Dataset>
Dimensions:              (alleles: 4, cohorts: 15, ploidy: 2, samples: 1142, variants: 57837885, windows: 289190)
Dimensions without coordinates: alleles, cohorts, ploidy, samples, variants, windows
Data variables:
    window_start         (windows) int64 0 200 400 ... 57837600 57837800
    window_stop          (windows) int64 200 400 600 ... 57837800 57837885
    call_genotype        (variants, samples, ploidy) int8 dask.array<chunksize=(65536, 1142, 2), meta=np.ndarray>
    call_genotype_mask   (variants, samples, ploidy) bool dask.array<chunksize=(65536, 1142, 2), meta=np.ndarray>
    sample_id            (samples) <U1 dask.array<chunksize=(1142,), meta=np.ndarray>
    variant_allele       (variants, alleles) |S1 dask.array<chunksize=(65536, 4), meta=np.ndarray>
    variant_contig       (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_position     (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    cohort_allele_count  (variants, cohorts, alleles) int32 dask.array<chunksize=(65536, 15, 4), meta=np.ndarray>

In [10]:
div = sg.diversity(ds, merge=False)
div

<xarray.Dataset>
Dimensions:         (cohorts: 15, windows: 289190)
Dimensions without coordinates: cohorts, windows
Data variables:
    stat_diversity  (windows, cohorts) float64 dask.array<chunksize=(328, 15), meta=np.ndarray>

In [11]:
div = div.chunk({"windows": 65536})
div

<xarray.Dataset>
Dimensions:         (cohorts: 15, windows: 289190)
Dimensions without coordinates: cohorts, windows
Data variables:
    stat_diversity  (windows, cohorts) float64 dask.array<chunksize=(65536, 15), meta=np.ndarray>

In [13]:
with ProgressBar():
    div.to_zarr(str(here() / 'data/sgkit/ag1000g_diversity.zarr'))

[########################################] | 100% Completed |  2min 34.7s
